In [3]:
#Import necessary libraries
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

In [ ]:
import pydicom
import numpy as np
from tensorflow.keras.models import model_from_json

# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data

def check_dicom(filename): 
    try:
        ds = pydicom.dcmread(filename)
        # Check modality is X-ray (DX)
        if ds.Modality != 'DX':
            print(f"File {filename}: Not an X-ray (Modality={ds.Modality})")
            return None
            
        # Check body part is chest
        if hasattr(ds, 'BodyPartExamined') and ds.BodyPartExamined.lower() != 'chest':
            print(f"File {filename}: Not a chest X-ray (BodyPartExamined={ds.BodyPartExamined})")
            return None
            
        # Check image position (if available)
        if hasattr(ds, 'PatientPosition'):
            print(f"Image Position: {ds.PatientPosition}")
            if ds.PatientPosition not in ['PA', 'AP']:
                print(f"Warning: Unexpected patient position: {ds.PatientPosition}")
                
        # Additional DICOM info for validation
        print(f"Image Type: {ds.get('ImageType', 'Not specified')}")
        print(f"Patient Position: {ds.get('PatientPosition', 'Not specified')}")
            
        img = ds.pixel_array
        return img
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean, img_std, img_size): 
    # Resize image if needed
    from cv2 import resize, INTER_LINEAR
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)  # Convert grayscale to 3-channel
    img_resized = resize(img, (img_size[2], img_size[1]), interpolation=INTER_LINEAR)
    img_resized = img_resized.astype(np.float32)
    # Normalize
    img_norm = (img_resized - img_mean) / img_std
    # Add batch dimension
    proc_img = np.expand_dims(img_norm, axis=0)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # Load model architecture
    with open(model_path, "r") as json_file:
        loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)
    # Load weights
    model.load_weights(weight_path)
    # Compile (use same loss/metrics as training)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prob = model.predict(img)[0][0]
    prediction = int(prob > thresh)
    return prediction

In [5]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model_architecture.json"      # Path to saved model architecture
weight_path = "xray_class_my_model.best.hdf5"  # Path to saved best weights

IMG_SIZE = (1, 224, 224, 3)  # Adjust if your model uses a different input size

# Load the mean and std used during training (replace with your actual values or load from file)
img_mean = 128.0  # Example: replace with your training mean
img_std = 64.0    # Example: replace with your training std

# Load the trained model
my_model = load_model(model_path, weight_path)

# Set the threshold chosen during model evaluation
thresh = 0.5  # Example: replace with your chosen threshold

# Use the .dcm files to test your prediction
for i in test_dicoms:
    img = check_dicom(i)
    if img is None:
        print(f"Skipping {i}: not a valid chest X-ray.")
        continue
    img_proc = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    pred = predict_image(my_model, img_proc, thresh)
    print(f"{i}: Prediction = {pred}")

FileNotFoundError: [Errno 2] No such file or directory: 'my_model_architecture.json'